In [2]:
import cv2
from keras.applications.resnet import preprocess_input as resnet_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.models import load_model
import keras
import numpy as np 
import pickle
import matplotlib.pyplot as plt 
import pandas as pd
import os
from sklearn.metrics import classification_report, accuracy_score, log_loss

In [3]:
DIR_TEST = "dataset-cropped/Test/"

In [4]:
images = []
labels = []

df = pd.read_csv("dataset-cropped/manual_ans.csv")

for tps in os.listdir(DIR_TEST):
    for paslon in os.listdir(os.path.join(DIR_TEST, tps)):
        for section in os.listdir(os.path.join(DIR_TEST, tps, paslon)):
            path = os.path.join(DIR_TEST, tps, paslon, section)

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (40, 240))
            img = cv2.GaussianBlur(img, (5, 5), 0)
            # imt = cv2.medianBlur(img, 5)

            th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 19, 2)
            # cv2.floodFill(th, None, (0, 0), 123, )
            
            th = th / 255.0
            images.append(th)
    
    curr_tps = df[df['TPS'] == tps]
    for col in (['Paslon_1', 'Paslon_2', 'Paslon_3']):
        label_str = '{:03d}'.format(curr_tps[col].values[0])
        labels.extend([int(n) for n in label_str])

In [5]:
def evaluation(model_file, images, labels, traditional=False):
    if traditional:
        with open(f'models/{model_file}', 'rb') as file:
            model = pickle.load(file)
    else:
        model = load_model(f'models/{model_file}')

    tmp_images = np.array(images)
    tmp_labels = np.array(labels)

    if traditional:
        tmp_images = tmp_images.reshape(tmp_images.shape[0], -1)
    else:
        tmp_images = np.repeat(tmp_images[..., np.newaxis], 3, -1)
        if model_file == 'ResNet50PreprocessInput':
            tmp_images = resnet_preprocess_input(tmp_images)
        elif model_file == 'VGG19':
            tmp_images = vgg19_preprocess_input(tmp_images)
        
    tmp_labels = keras.utils.to_categorical(tmp_labels, num_classes=10)

    if traditional:
        y_pred = model.predict(tmp_images)
        y_prob = model.predict_proba(tmp_images)
        y_true = np.argmax(tmp_labels, axis=1)

        loss = log_loss(tmp_labels, y_prob)
        acc = accuracy_score(y_true, y_pred)
        print(f'Loss: {loss}')
        print(f'Accuracy: {acc}')
    else:
        loss, acc = model.evaluate(tmp_images, tmp_labels)
        print(f'Loss: {loss}')
        print(f'Accuracy: {acc}')
        y_pred = model.predict(tmp_images)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = np.argmax(tmp_labels, axis=1)

    print(classification_report(y_true, y_pred))

RESNET50

In [6]:
evaluation('ResNet50PreprocessInput', images, labels)

51/51 [==============================] - 4s 24ms/step - loss: 0.0643 - accuracy: 0.9858
Loss: 0.06432803720235825
Accuracy: 0.9858024716377258
51/51 [==============================] - 2s 20ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       528
           1       1.00      0.99      0.99       289
           2       0.99      1.00      1.00       133
           3       1.00      0.98      0.99       120
           4       1.00      1.00      1.00       107
           5       0.99      0.97      0.98        96
           6       0.97      0.97      0.97        90
           7       0.98      0.98      0.98        97
           8       0.98      0.94      0.96        84
           9       0.99      0.96      0.97        76

    accuracy                           0.99      1620
   macro avg       0.99      0.98      0.98      1620
weighted avg       0.99      0.99      0.99      1620



In [7]:
evaluation('ResNet50', images, labels)

51/51 [==============================] - 2s 25ms/step - loss: 0.0717 - accuracy: 0.9858
Loss: 0.0717252865433693
Accuracy: 0.9858024716377258
51/51 [==============================] - 2s 20ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       528
           1       0.98      1.00      0.99       289
           2       0.98      0.99      0.99       133
           3       1.00      0.96      0.98       120
           4       0.99      0.98      0.99       107
           5       0.98      0.98      0.98        96
           6       0.98      0.99      0.98        90
           7       1.00      1.00      1.00        97
           8       0.98      0.96      0.97        84
           9       0.97      0.97      0.97        76

    accuracy                           0.99      1620
   macro avg       0.98      0.98      0.98      1620
weighted avg       0.99      0.99      0.99      1620



VGG19

In [8]:
evaluation('VGG19', images, labels)

51/51 [==============================] - 4s 50ms/step - loss: 0.1767 - accuracy: 0.9698
Loss: 0.1767251342535019
Accuracy: 0.969753086566925
51/51 [==============================] - 2s 33ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       528
           1       0.99      0.98      0.98       289
           2       1.00      0.98      0.99       133
           3       0.98      0.97      0.97       120
           4       0.98      0.97      0.98       107
           5       0.94      0.95      0.94        96
           6       0.98      0.96      0.97        90
           7       0.98      0.95      0.96        97
           8       0.93      0.93      0.93        84
           9       0.99      0.88      0.93        76

    accuracy                           0.97      1620
   macro avg       0.97      0.96      0.96      1620
weighted avg       0.97      0.97      0.97      1620



RANDOM FOREST CLASSIFIER

In [9]:
evaluation('RandomForest.pkl', images, labels, traditional=True)

Loss: 0.22675441273368813
Accuracy: 0.9882716049382716
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       528
           1       1.00      1.00      1.00       289
           2       0.99      1.00      0.99       133
           3       1.00      0.99      1.00       120
           4       0.99      0.99      0.99       107
           5       0.98      0.98      0.98        96
           6       0.96      1.00      0.98        90
           7       0.99      0.98      0.98        97
           8       0.99      0.96      0.98        84
           9       0.99      0.91      0.95        76

    accuracy                           0.99      1620
   macro avg       0.99      0.98      0.98      1620
weighted avg       0.99      0.99      0.99      1620



SVM

In [10]:
evaluation('SVM.pkl', images, labels, traditional=True)

Loss: 0.050471439057845985
Accuracy: 0.9925925925925926
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       528
           1       1.00      1.00      1.00       289
           2       1.00      1.00      1.00       133
           3       1.00      0.99      1.00       120
           4       1.00      1.00      1.00       107
           5       1.00      0.98      0.99        96
           6       0.98      1.00      0.99        90
           7       1.00      0.99      0.99        97
           8       0.99      0.96      0.98        84
           9       0.99      0.96      0.97        76

    accuracy                           0.99      1620
   macro avg       0.99      0.99      0.99      1620
weighted avg       0.99      0.99      0.99      1620

